In [1]:
import gdn_datacollector
import datetime
import gdn_db
from googleads import adwords
import pandas as pd
import numpy as np
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
CRITERIA_LIST = ['ADGROUP', 'URL', 'CRITERIA', 'AGE_RANGE', 'DISPLAY_KEYWORD', 'AUDIENCE']

In [109]:
DATABASE_INDEX = {
    'ADGROUP': 'adgroup_insights',
    'URL': 'url_insights',
    'CRITERIA': 'criteria_insights',
    'AUDIENCE': 'audience_insights',
    'AGE_RANGE': 'age_range_insights',
    'DISPLAY_KEYWORD': 'display_keyword_insights',
}
SCORE_INDEX = {
    'ADGROUP': 'adgroup',
    'URL': 'url',
    'CRITERIA': 'criteria',
    'AUDIENCE': 'audience',
    'AGE_RANGE': 'age_range',
    'DISPLAY_KEYWORD': 'display_keyword',
}

In [3]:
def get_performance_insights(client, campaign_id=None, adgroup_id=None, date_preset=None, performance_type='ADGROUP'):
    # URL
    URL_FIELDS_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'AdGroupStatus', 'Cost', 'AverageCost',
        'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion', 'DisplayName', ]

    # CRITERIA,
    CRITERIA_FIELDS_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
        'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

    # AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
    BIDDABLE_FIELDS_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
        'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

    # ADGROUP
    ADGROUP_FIELDS_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'CpmBid', 'CpvBid', 'CpcBid', 'TargetCpa',
        'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

    NON_NUMERIC_FIELDS_LIST = [
        'Ad group type', 'Ad group state', 'Bid Strategy Type', 'Keyword', 'Age Range', 'Audience', 'Keyword / Placement', 'Criteria Display Name']

    NUMERIC_FIELDS_LIST = [
        'Max. CPM', 'Max. CPV', 'Max. CPC', 'Default max. CPC', 'Target CPA', 'Cost', 'Avg. Cost', 'Avg. CPC', 'Cost / conv.']



    ADGROUP_COLUMNS_LIST  = [
        'customer_id', 'campaign_id', 'channel_type', 'adgroup_id', 'status', 'cpm_bid', 'cpv_bid', 'cpc_bid', 'cpa_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    URL_COLUMNS_LIST      = [
        'customer_id', 'campaign_id', 'adgroup_id', 'status', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion', 'url_display_name']
    CRITERIA_COLUMNS_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'keyword_placement', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    AUDIENCE_COLUMNS_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'audience', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    AGE_RANGE_COLUMNS_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'age_range', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    BIDDABLE_COLUMNS_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']

    FIELD_INDEX = {
        'ADGROUP': ADGROUP_FIELDS_LIST,
        'URL': URL_FIELDS_LIST,
        'CRITERIA': CRITERIA_FIELDS_LIST,
        'AUDIENCE': BIDDABLE_FIELDS_LIST,
        'AGE_RANGE': BIDDABLE_FIELDS_LIST,
        'DISPLAY_KEYWORD': BIDDABLE_FIELDS_LIST,
    }
    COLUMN_INDEX = {
        'ADGROUP': ADGROUP_COLUMNS_LIST,
        'URL': URL_COLUMNS_LIST,
        'CRITERIA': CRITERIA_COLUMNS_LIST,
        'AUDIENCE': AUDIENCE_COLUMNS_LIST,
        'AGE_RANGE': AGE_RANGE_COLUMNS_LIST,
        'DISPLAY_KEYWORD': BIDDABLE_COLUMNS_LIST,
    }

    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id is not None and adgroup_id is None:
        operand = {
            'field': 'CampaignId',
            'operator': 'EQUALS',
            'values': [campaign_id]
        }
#         FIELD_INDEX[performance_type].remove('AdGroupId')
#         COLUMN_INDEX[performance_type].remove('adgroup_id')
        fields = FIELD_INDEX[performance_type]
        columns = COLUMN_INDEX[performance_type]
    elif adgroup_id is not None:
        operand = {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values': [adgroup_id]
        }
        fields = FIELD_INDEX[performance_type]
        columns = COLUMN_INDEX[performance_type]
        
    else:
        print('get_performance_insights: Missing arguments campaign_id or adgroup_id.')
        operand = None
        return
    
    report = {
        'downloadFormat': 'CSV',
        'reportName': performance_type+'_PERFORMANCE_REPORT',
        'reportType': performance_type+'_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'selector': {
            'fields': fields,
            'predicates': [operand]
        }
    }
    # Print out the report as a string
    with open(performance_type+'.csv', 'wb') as output_file:
        report_downloader.DownloadReport(
            report, output=output_file, skip_report_header=True, skip_column_header=False,
            skip_report_summary=True, include_zero_impressions=False)
    with open(performance_type+'.csv')as csv_file:
        df = pd.read_csv(csv_file, sep=",", quotechar='"')
        
        df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )] = df[df.columns.difference( NON_NUMERIC_FIELDS_LIST )].apply(pd.to_numeric, errors='coerce')
        
        df[df.columns.intersection( NUMERIC_FIELDS_LIST )] = df[df.columns.intersection( NUMERIC_FIELDS_LIST )].div(1000000)
        
        df.columns = columns
        df.sort_values(by=['impressions'], ascending=False).reset_index(drop=True)
        gdn_db.into_table( df, DATABASE_INDEX[performance_type] )
        return df


    

In [4]:
def retrive_all_criteria_insights():
    starttime = datetime.datetime.now()
    df_camp = gdn_db.get_campaign()
    campaign_id_list = df_camp['campaign_id'].unique()
    
    # retrive all criteria insights
    for campaign_id in campaign_id_list:
        df = df_camp[df_camp.campaign_id==campaign_id]
        customer_id = df['customer_id'].iloc[0]
        destination_type = df['destination_type'].iloc[0]
#         adgroup_id_list = gdn_datacollector.Campaign( customer_id, campaign_id, destination_type).get_adgroup_id_list()        

        for criteria in CRITERIA_LIST:
            get_performance_insights(client, campaign_id=campaign_id, performance_type=criteria)
            

In [12]:
%%time
from tabulate import tabulate
df_camp = gdn_db.get_campaign()
campaign_id_list = df_camp['campaign_id'].unique()
for campaign_id in campaign_id_list:
    df = df_camp[df_camp.campaign_id==campaign_id]
    customer_id = df['customer_id'].iloc[0]
    destination_type = df['destination_type'].iloc[0]
    client.SetClientCustomerId(customer_id)
    for criteria in CRITERIA_LIST:
        df = get_performance_insights(client, campaign_id=1777436620, performance_type=criteria).head(1)
#         print(tabulate(df, headers=df.columns, tablefmt='simple'))


CPU times: user 260 ms, sys: 19.5 ms, total: 280 ms
Wall time: 16.1 s


In [6]:
%%time
# import gdn_db
# df_camp = gdn_db.get_campaign()
# campaign_id_list = df_camp['campaign_id'].unique()
# for campaign_id in campaign_id_list:
#     for criteria in CRITERIA_LIST:
# table = DATABASE_INDEX["CRITERIA"]
# df = gdn_db.get_table(campaign_id=1747836664, table=table)
# print(tabulate(df.head(1), headers=df.columns, tablefmt='simple'))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [7]:
def ga_optimal_weight(campaign_id):
    request_time = datetime.datetime.now().date()
    mydb = gdn_db.connectDB("dev_gdn")
    df_weight = pd.read_sql(
        "SELECT * FROM optimal_weight WHERE campaign_id=%s " % (campaign_id), con=mydb)
    df_camp = pd.read_sql(
        "SELECT * FROM campaign_target WHERE campaign_id=%s " % (campaign_id), con=mydb)
    df_adgroup = pd.read_sql(
        "SELECT * FROM adgroup_initial_bid WHERE campaign_id=%s " % (campaign_id), con=mydb)
    destination_type = df_camp['destination_type'].iloc[0]
    adgroup_list = df_adgroup['adgroup_id'].unique()
    for adgroup_id in adgroup_list:

        df = ObjectiveFunc().adgroup_status(adgroup_id)
        r = ObjectiveFunc.adgroup_fitness(df_weight, df)

        df_final = pd.DataFrame({'campaign_id': campaign_id, 'adgroup_id': adgroup_id,
                                 'score': r, 'request_time': request_time}, index=[0])
        print(adgroup_id, df_final['score'].iloc[0])
        gdn_db.into_table(df_final, table="adgroup_score")

    mydb.close()
    return

In [8]:
def adgroup_status(self, adgroup_id):
    df = pd.DataFrame({'adgroup_id': [], 'target': [],
                       'impressions': [], 'bid_amount': []})
    df_adgroup = pd.read_sql(
        "SELECT * FROM adgroup_insights WHERE adgroup_id=%s ORDER BY request_time DESC LIMIT 1" % (adgroup_id), con=self.mydb)
    df_camp = pd.read_sql("SELECT * FROM campaign_target WHERE campaign_id=%s" %
                          (df_adgroup['campaign_id'].iloc[0]), con=self.mydb)
    df_adgroup['daily_budget'] = df_camp['daily_budget']
    df_adgroup['bid_amount'] = df_adgroup[ BIDDING_INDEX[df_adgroup['bidding_type'].iloc[0]] ]
    df_adgroup['target'] = df_adgroup[ TARGET_INDEX[df_adgroup['bidding_type'].iloc[0]] ]
    df_camp['daily_destination'] = df_camp['destination'] / df_camp['period']
    df_temp = pd.merge(
        df_adgroup[['campaign_id', 'adgroup_id', 'daily_budget', 'impressions']],
        df_adgroup[['adgroup_id', 'spend', 'bid_amount',
                    'target', 'cost_per_target']], on=['adgroup_id']
    )
    df_status = pd.merge(
        df_temp,
        df_camp[['campaign_id', 'daily_destination']], on=['campaign_id']
    )
#         df_status = df_temp
    df = pd.concat([df, df_status], ignore_index=True, sort=True)
    self.mydb.close()
    return df

In [115]:
import gdn_db
BIDDING_INDEX = {
    'cpc': 'cpc_bid',
    'cpa': 'cpa_bid',
}
TARGET_INDEX = {
    'cpc': 'clicks',
    'cpa': 'conversions',
}

def criteria_status( campaign_id=None, adgroup_id=None, criteria=None):
    mydb = gdn_db.connectDB("dev_gdn")
    df = pd.DataFrame({'adgroup_id': [], 'target': [],
                       'impressions': [], 'bid_amount': []})
    df_camp = pd.read_sql(
        "SELECT * FROM campaign_target WHERE campaign_id='{}'".format(campaign_id), con=mydb)    
#     for criteria in CRITERIA_LIST:
    table = DATABASE_INDEX[criteria]
    df = pd.read_sql(
        "SELECT * FROM {} WHERE campaign_id='{}'".format(table, campaign_id), con=mydb)
    df_weight = pd.read_sql(
        "SELECT * FROM optimal_weight WHERE campaign_id='{}' ".format(campaign_id), con=mydb)
    if not df.empty:
        if criteria != "URL" and criteria != "CRITERIA":
            print(criteria)
            df['daily_budget'] = df_camp['daily_budget'].iloc[0]
            df['bid_amount'] = df[ BIDDING_INDEX[ df['bidding_type'].iloc[0] ] ]
            df['target'] = df[ TARGET_INDEX[ df['bidding_type'].iloc[0] ] ]
            daily_destination = df_camp['destination'] / df_camp['period']
            df['daily_destination'] = daily_destination.iloc[0]
            for index, row in df.iterrows():
                df_final = pd.DataFrame(data=[row], columns=df.columns, index=[0])
                r = ObjectiveFunc.adgroup_fitness(df_weight, df_final)
                
#                 df_final = pd.DataFrame({
#                     'campaign_id':campaign_id,
#                     'keyword': row['keyword'], 'keyword_id': row['keyword_id'], 'score': r}, index=[0])
                print(df_final.columns, r[0])
#                 gdn_db.into_table(df_final, table=SCORE_INDEX[criteria]+"_score")
            
            
            
            
            mydb.close()
            return 


In [116]:
criteria_status(campaign_id=1777436620, criteria="DISPLAY_KEYWORD")

DISPLAY_KEYWORD
Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_click', 'cost_per_conversion', 'request_time', 'daily_budget',
       'bid_amount', 'target', 'daily_destination'],
      dtype='object') -4.669226088078336
Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_click', 'cost_per_conversion', 'request_time', 'daily_budget',
       'bid_amount', 'target', 'daily_destination'],
      dtype='object') -4.669226088078336
Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_c

Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_click', 'cost_per_conversion', 'request_time', 'daily_budget',
       'bid_amount', 'target', 'daily_destination'],
      dtype='object') -4.669226088078336
Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_click', 'cost_per_conversion', 'request_time', 'daily_budget',
       'bid_amount', 'target', 'daily_destination'],
      dtype='object') -4.669226088078336
Index(['customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id',
       'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend',
       'cost_per_target', 'impressions', 'clicks', 'conversions',
       'cost_per_click', 'cost_per

In [15]:
from genetic_algorithm_gdn import *
campaign_id = 1777436620
global df
df = ObjectiveFunc().campaign_status(campaign_id)
bound = np.tile([[0], [1]], vardim)
ga = GeneticAlgorithm(sizepop, vardim, bound, MAXGEN, params)
optimal = ga.solve()
score = ObjectiveFunc.fitnessfunc(optimal, df)

score_columns=['weight_kpi', 'weight_spend', 'weight_bid']
df_score = pd.DataFrame(data=[optimal], columns=['weight_kpi', 'weight_spend', 'weight_bid'], index=[0])
#         score_columns=['weight_kpi', 'weight_spend', 'weight_bid', 'weight_width']
#         df_score = pd.DataFrame(data=[optimal], columns=['weight_kpi', 'weight_spend', 'weight_bid', 'weight_width'], index=[0])        

df_final = pd.DataFrame({'campaign_id':campaign_id, 'score':score}, columns=['campaign_id', 'score'], index=[0])
df_final = pd.concat( [df_score, df_final], axis=1, sort=True, ignore_index=False)

print(df_final)
gdn_db.check_optimal_weight(campaign_id, df_final)

/home/tim_su/ai_optimizer/opt/ai_optimizer/ai_optimizer/codes/genetic_algorithm_gdn.py:290: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)


NameError: name 'df' is not defined

In [16]:
df

,campaign_id,campaign_cpc,campaign_target,impressions,campaign_bid,spend,daily_budget,daily_destination,budget_per_day
0,1777436620,3.59448,210,106051,4.166667,754.842,250.0,60.0,250.0
